<a href="https://colab.research.google.com/github/kabilan942/Natural-Language-Processing/blob/main/Research-Work/Legal/Transformers/LDS_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.21.0
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
!pip install rouge
nltk.download('punkt')
from rouge import Rouge
!pip install datasets
from datasets import Dataset, DatasetDict
!pip install rouge_score
!pip install datasets evaluate transformers[sentencepiece]
!pip install nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import glob
import nltk
nltk.download('punkt')
from nltk import tokenize
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F
import torch
import json
import random
import os
import sys
sys.path.insert(0, '../')
from IPython.display import display, HTML
import torch
from datasets import load_dataset, load_metric, Dataset
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
rouge_score = evaluate.load('rouge')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=85ef7265046cb778f5e23e8c52102521f55a0365bf46b21f6ca4ad6f9d36e8ae
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##CLS Chunking

In [ ]:
from transformers import BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = False)
bert_model.eval()

**Have to change path**

In [ ]:
def get_sen_encoding(sents):
    '''
    Function to generate encoding for each word in the input list
    input: sents - List of sentences
    returns the list of the sentence encoding
    '''
    a = 0.001
    answer = None
    for sent in sents:
        ip =tokenizer(sent, return_tensors='pt', max_length=150, truncation=True, padding='max_length')  # to convert input text into integer ids
        tokens = tokenizer.convert_ids_to_tokens(ip['input_ids'][0])  # converts id into tokens with inclusion of special tokens added during tokenization such as [CLS]
        bert_output = bert_model(**ip)  # BertModel;
        embedding = bert_output['pooler_output'].clone().detach()
        if answer == None:
            answer = embedding
            answer.resize_(1, 768)
        else:
            embedding.resize_(1, 768)
            answer = torch.cat((answer, embedding),0)
    return answer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity_l_l(l1, l2):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary
    '''
    l = l1+l2  # list of sentences in both summary and document
    sents_encodings = get_sen_encoding(l)   # return a list of tensors of length 768 each, for each element of the list
    similarities=cosine_similarity(sents_encodings)   # calculates pairwise similiarities with elements in list if 2 arguments are not passed in; notice along diagonal its almost always equal to 1; https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity

    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    return result

In [ ]:
def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in split_to_sentences(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

In [ ]:
def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents


def get_chunks_data_from_docV2(doc, summ):
    '''
    Function to generate chunks along with their summaries
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries
    '''
    chunk_summ_word_threshold = 25
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)

    result = (similarity_l_l(summ_sents,doc_sents))

    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]

    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 465):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(tokenizer.tokenize(summ)) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ

In [ ]:
# for training dataset

def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/train-data/"
    return path



In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Single/"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary


In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


2
2
2


In [ ]:
import pandas as pd
from tqdm import tqdm
training_chunks = []
training_summs = []
doc_id = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
    for j in range(len(cks)):
      doc_id.append(names[i])
df = pd.DataFrame({'data':training_chunks,'summary':training_summs,'doc_id':doc_id})
df.to_excel('FD_CLS_512_train.xlsx')

100%|██████████| 2/2 [03:19<00:00, 99.59s/it]


In [ ]:
# for test dataset

def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary


In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))

In [ ]:
import pandas as pd
from tqdm import tqdm
training_chunks = []
training_summs = []
doc_id = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
    for j in range(len(cks)):
      doc_id.append(na mes[i])
df = pd.DataFrame({'data':training_chunks,'summary':training_summs,'doc_id':doc_id})
df.to_excel('FD_CLS_512_test.xlsx')

##Validation of pre-trained Model

**Have to change path**

In [ ]:
filepath = '/content/FD_CLS_512_test.xlsx'
test_df = pd.read_excel(filepath,usecols='B,C')

In [ ]:
test_df.shape

(54, 2)

In [ ]:
test_df.head()

,data,summary
0,"Sub sec tion (2) however provided that ""nothin...",The charge in respect of urban immoveable prop...
1,72 of 1950. Petition under article 32 of the C...,"Per PATANJALI SASTRI, MUKHERJEA and DAS JJ.In ..."
2,The petitioner 's case is that the shareholder...,"An incorporated company, therefore, can come u..."
3,"Supposing, howev er, that the impugned Act was...",If on the face of the law there is no classifi...
4,The powers and the duties of the directors are...,The company was made a respondent and opposed ...


In [ ]:
def model_load(checkpoint):
     #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def sum_generate(FileContent,tokenizer,model):
    inputs = tokenizer(FileContent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  checkpoint = "t5-small"
  result_rouge_list = []
  for i in range(test_df.shape[0]):
    FileContent,summaryContent=test_df['data'][i], test_df['summary'][i]
    tokenizer,model=model_load(checkpoint)
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1
tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1


In [ ]:
len(result_rouge_list)

3

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.5023809523809524, 'p': 0.04105263157894737, 'f': 0.07580378809495004}, 'rouge-2': {'r': 0.23511904761904762, 'p': 0.012944856226672975, 'f': 0.024313052586233456}, 'rouge-l': {'r': 0.4547619047619048, 'p': 0.03754385964912281, 'f': 0.06926784038253174}}


## Fine-tuning T5 using CLS

In [ ]:
filepath = '/content/FD_CLS_512 (2).xlsx'
df = pd.read_excel(filepath,usecols='B,C')

In [ ]:
df.shape

(7, 2)

In [ ]:
df.head()

,data,summary
0,The decision of the point depends firstly on t...,"The expression ""capital charge"" in s.9(1) (iv)..."
1,Urban immove able property tax is leviable und...,secure the discharge of a liability of a capi ...
2,The law as to them varies from province to pro...,The charge in respect of urban immoveable prop...
3,XXIX of 1950. Application under article 32 of ...,(9.)(FAZL ALI J. dissenting) that inasmuch as ...
4,Section 7 (1) (c) under which the aforesaid or...,Section 7 (1) (c) of the East Punjab Public Sa...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.20,shuffle=False)

In [ ]:
train_df.shape,valid_df.shape

((5, 2), (2, 2))

In [ ]:
valid_df.reset_index(inplace=True)
valid_df.drop('index',inplace=True,axis=1)

In [ ]:
valid_df.head()

,data,summary
5,"The petitioners, on whose behalf this provisio...",Public disorders which disturb the public tran...
6,"In order to understand the scope of the Act, i...",Romesh Thappar vs The State ([1950] S.C.R.Held...


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(valid_df)

In [ ]:
train_dataset

Dataset({
    features: ['data', 'summary', '__index_level_0__'],
    num_rows: 5
})

In [ ]:
train_df.shape

(5, 2)

In [ ]:
len(train_df['data'][0].split())

461

In [ ]:
datasets = DatasetDict()
datasets['train'] = train_dataset.remove_columns('__index_level_0__')
datasets['valid'] = val_dataset

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['data', 'summary'],
        num_rows: 5
    })
    valid: Dataset({
        features: ['data', 'summary', '__index_level_0__'],
        num_rows: 2
    })
})

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.d67b370cd9d75f81ad4eb421ee7b8db09e0b6a6c693b8c2b423af5d7bcac6205
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32

In [ ]:
max_input_length = 512
max_target_length = 256

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['data'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (highlights)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('highlights') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_dataset = datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
    valid: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Seq2SeqLM: sequence-to-sequence language modeling head

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # model_checkpoint = "t5-small"

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.d67b370cd9d75f81ad4eb421ee7b8db09e0b6a6c693b8c2b423af5d7bcac6205
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "len

In [ ]:
# parameters of Seq2SeqTrainingArguments
batch_size = 2
num_training_epochs = 20
logging_steps = len(tokenized_dataset["train"])//2
model_name = "t5-small-finetuned-cls-part"
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5"

args = Seq2SeqTrainingArguments(evaluation_strategy='steps',
                                learning_rate=5.6e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                eval_steps=2,
                                save_steps=2,
                                save_total_limit=3,
                                num_train_epochs=num_training_epochs,
                                predict_with_generate=True,
                                logging_steps=logging_steps,
                                output_dir="t5-small-finetuned-cls-part",
                                load_best_model_at_end=True,
                                metric_for_best_model='eval_loss',
                                greater_is_better=False,
                                push_to_hub=False)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import evaluate
rouge_score = evaluate.load('rouge')
def compute_metric(eval_pred):
  predictions, labels = eval_pred
  # Decode generated summaries into text (summaries generated from the review_body)
  # batch_decode -> Returns List[str]; The list of decoded sentences; Convert a list of lists of token ids into a list of strings by calling decode.
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # skip_special_tokens - Whether or not to remove special tokens in the decoding.
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  # Decode reference summaries into text ('review_title')
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # ROUGE expects a newline after each sentence
  # strip() method removes characters from both left and right based on the argument
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_labels]
  # Computing ROUGE score
  # Stemming is used to remove plurals and word suffixes such as (ing, ion, ment)
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v,4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
datasets['train'].column_names, len(datasets['train'].column_names)

(['data', 'summary'], 2)

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(datasets['train'].column_names)
tokenized_dataset['train'].column_names

['input_ids', 'attention_mask', 'labels']

In [ ]:
from transformers import EarlyStoppingCallback
# args = Seq2SeqTrainingArguments()
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
trainer = Seq2SeqTrainer(model, args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["valid"],
                         data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metric,
                         callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 60


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
2,0.131800,5.333674,12.196800,3.539800,9.588100,10.457700
4,0.356200,5.392924,12.196800,3.539800,9.588100,10.457700
6,0.337900,5.419914,12.196800,3.539800,9.588100,10.457700
8,0.560000,5.448959,12.196800,3.539800,9.588100,10.457700
10,0.449500,5.465497,12.196800,3.539800,9.588100,10.457700
12,0.594000,5.479875,12.196800,3.539800,9.588100,10.457700
14,0.675400,5.487410,12.196800,3.539800,9.588100,10.457700
16,0.852400,5.491920,12.196800,3.539800,9.588100,10.457700
18,0.463200,5.495315,12.196800,3.539800,9.588100,10.457700
20,0.602100,5.502688,12.196800,3.539800,9.588100,10.457700


***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2
  Batch size

KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2
  Batch size = 2


{'eval_loss': 4.917222023010254,
 'eval_rouge1': 12.1968,
 'eval_rouge2': 3.5398,
 'eval_rougeL': 9.5881,
 'eval_rougeLsum': 10.4577,
 'eval_runtime': 0.4051,
 'eval_samples_per_second': 4.938,
 'eval_steps_per_second': 2.469,
 'epoch': 20.0}

**Have to change path**

In [ ]:
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5"
trainer.save_model(model_dir)

Saving model checkpoint to /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5
Configuration saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5/config.json
Model weights saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5/special_tokens_map.json
Copy vocab file to /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5/spiece.model


##Validation of fine-tuned Model

In [ ]:
filepath = '/content/FD_CLS_512_test.xlsx'
test_df = pd.read_excel(filepath,usecols='B,C')

FileNotFoundError: ignored

In [ ]:
def model_load(checkpoint):
     #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def sum_generate(FileContent,tokenizer,model):
    inputs = tokenizer(FileContent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

**Have to change path**

In [ ]:
# inputs to the model
if __name__ == "__main__":
  checkpoint = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5"
  result_rouge_list = []
  for i in range(test_df.shape[0]):
    FileContent,summaryContent=test_df['data'][i], test_df['summary'][i]
    tokenizer,model=model_load(checkpoint)
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

In [ ]:
len(result_rouge_list)

15

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.33020202020202016, 'p': 0.04780316310364701, 'f': 0.0820922766921572}, 'rouge-2': {'r': 0.08127058127058126, 'p': 0.007673922410429862, 'f': 0.01379944906931314}, 'rouge-l': {'r': 0.31858585858585853, 'p': 0.04555700642501312, 'f': 0.07834985831559248}}
